In [ ]:
!pip install langchain_openai

In [ ]:
!pip install langchain_community

In [ ]:
!pip install faiss-cpu

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## Retrieval Augmented Generation (RAG)

### Carregando Documentos - Loading

In [ ]:
# https://python.langchain.com/docs/how_to/document_loader_html/
# https://python.langchain.com/docs/integrations/document_loaders/

import bs4
from langchain_community.document_loaders import WebBaseLoader

# Filtra o conteúdo da página por uma classe específica
bs4_strainer = bs4.SoupStrainer(class_=("container-wrapper"))

# Carrega o conteúdo da página
loader = WebBaseLoader(
    web_paths=("https://cesar.breezy.hr/p/00f79174d8ad-pesquisador-em-inteligencia-artificial-e-sistemas-distribuidos",),
    bs_kwargs={"parse_only": bs4_strainer},
)

# Carrega o conteúdo da página
docs = loader.load()

len(docs[0].page_content)

4301

### Dividindo Documentos - Splitting/Chunking

In [ ]:
# https://python.langchain.com/v0.2/docs/how_to/#text-splitters

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=500, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

6

In [ ]:
print(all_splits[3].page_content)

Requisitos e Qualificações:
Doutorado em áreas correlatas;Compreenda e desenvolva modelos de machine learning e deep learning para resolver desafios complexos de cibersegurança;Conhecimento em frameworks de machine learning como TensorFlow, PyTorch ou scikit-learn para desenvolver modelos preditivos e de detecção de anomalias aplicados à cibersegurança;Entenda a arquitetura distribuída dos sistemas e garanta a integração eficiente de soluções de IA com aplicações em cloud;Habilidade em manipulação e visualização de dados com Pandas, NumPy, Matplotlib e Seaborn para explorar grandes volumes de dados;Experiência com AWS, Google Cloud ou Azure para projetar e implementar infraestruturas escaláveis e resilientes;Familiaridade com Kubernetes e Docker para garantir escalabilidade e resiliência de sistemas distribuídos;Experiência com controle de versão (Git) e repositórios remotos como GitLab;Inglês avançado para leitura, escrita e comunicação, facilitando a colaboração com equipes globais.


### Indexando - Store

In [ ]:
# https://python.langchain.com/v0.2/docs/how_to/embed_text/

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("precisa de doutorado para a vaga?")

len(retrieved_docs)

6

In [ ]:
print(retrieved_docs[0].page_content)

Requisitos e Qualificações:
Doutorado em áreas correlatas;Compreenda e desenvolva modelos de machine learning e deep learning para resolver desafios complexos de cibersegurança;Conhecimento em frameworks de machine learning como TensorFlow, PyTorch ou scikit-learn para desenvolver modelos preditivos e de detecção de anomalias aplicados à cibersegurança;Entenda a arquitetura distribuída dos sistemas e garanta a integração eficiente de soluções de IA com aplicações em cloud;Habilidade em manipulação e visualização de dados com Pandas, NumPy, Matplotlib e Seaborn para explorar grandes volumes de dados;Experiência com AWS, Google Cloud ou Azure para projetar e implementar infraestruturas escaláveis e resilientes;Familiaridade com Kubernetes e Docker para garantir escalabilidade e resiliência de sistemas distribuídos;Experiência com controle de versão (Git) e repositórios remotos como GitLab;Inglês avançado para leitura, escrita e comunicação, facilitando a colaboração com equipes globais.


### Buscando e Recuperando Informações - Retrieve

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [ ]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n', additional_kwargs={}, response_metadata={})]


### Gerando Respostas - Generate

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [ ]:
for chunk in rag_chain.stream("Tem plano de saúde como benefício?"):
    print(chunk, end="", flush=True)

Sim, o CESAR oferece plano de saúde como um dos benefícios para os colaboradores. Além disso, também há plano odontológico e outros auxílios.

## Exercícios

### Exercício 1
Faça um RAG com um pequeno arquivo de texto, contendo informações que, certamente, a LLM não conheça. Você deverá construir o arquivo e enviar para o ambiente de execução. Escolha a forma de chunking apropriada para o seu documento.

In [ ]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 41.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 6.8 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain_community.document_loaders import UnstructuredHTMLLoader

file_path = "/content/drive/Shareddrives/Projeto_final_rna_deep_learning/Funcionários da Companhia Zaffari fazem novas denúncias trabalhistas - ICL Notícias.html"

loader = UnstructuredHTMLLoader(file_path)
data = loader.load()

print(data)

[Document(metadata={'source': '/content/drive/Shareddrives/Projeto_final_rna_deep_learning/Funcionários da Companhia Zaffari fazem novas denúncias trabalhistas - ICL Notícias.html'}, page_content="Melhor site nacional de notícias (BPA)\n\nSeja Membro ICL\n\nSeja Membro ICL\n\nSociedade\n\nFuncionários da Companhia Zaffari fazem novas denúncias trabalhistas\n\nUma série de casos chegou na redação do BdF e do O Futuro após a repercussão nacional da primeira reportagem\n\n21/12/2024 | 08h30\n\nCompartilhe\n\nJornal O Futuro\n\nouça este conteúdo readme\n\n0:00 1.0x\n\nok\n\nPublicidade\n\nPor Theo Dalla — Brasil de Fato e Jornal O Futuro\n\nA publicação da reportagem sobre as condições de trabalho na Companhia Zaffari, fruto de uma parceria entre o Jornal O Futuro e o Brasil de Fato, abriu a porta para uma série de novas denúncias de atuais e ex-funcionários da empresa. Eles não só reafirmam as violações de direitos já denunciadas, como apontam novos problemas que reforçam a ideia de c

In [ ]:
# Carrega o conteúdo da página
docs = loader.load()

len(docs[0].page_content)

16290

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=15100, chunk_overlap=12000, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

2

In [ ]:
print(all_splits[0].page_content)

Melhor site nacional de notícias (BPA)

Seja Membro ICL

Seja Membro ICL

Sociedade

Funcionários da Companhia Zaffari fazem novas denúncias trabalhistas

Uma série de casos chegou na redação do BdF e do O Futuro após a repercussão nacional da primeira reportagem

21/12/2024 | 08h30

Compartilhe

Jornal O Futuro

ouça este conteúdo readme

0:00 1.0x

ok

Publicidade

Por Theo Dalla — Brasil de Fato e Jornal O Futuro

A publicação da reportagem sobre as condições de trabalho na Companhia Zaffari, fruto de uma parceria entre o Jornal O Futuro e o Brasil de Fato, abriu a porta para uma série de novas denúncias de atuais e ex-funcionários da empresa. Eles não só reafirmam as violações de direitos já denunciadas, como apontam novos problemas que reforçam a ideia de condições degradantes de trabalho: adolescentes trabalhando na escala 10×1 e por mais de 10 horas nos “dias de dobra”, constantes desvios de função, controle do uso do banheiro, constrangimentos e adoecimento mental.

Por outro l

In [ ]:
vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("Há abuso de poder por parte da empresa?")

len(retrieved_docs)

2

In [ ]:
print(retrieved_docs[0].page_content)

O Tribunal Superior do Trabalho (TST) tem o entendimento de que a restrição do uso do banheiro fere o princípio da dignidade humana e pode configurar “abuso do poder diretivo do empregador”. A decisão argumenta que “o Tribunal Superior do Trabalho tem entendido que a restrição pelo empregador ao uso de banheiro pelos seus empregados fere o princípio da dignidade da pessoa humana, tutelado no artigo 1º, inciso III, da Constituição Federal, traduzindo-se em verdadeiro abuso no exercício do poder diretivo da empresa (artigo 2º da CLT), o que configura ato ilícito, sendo, assim, indenizável o dano moral sofrido pelos empregados.”

A decisão também enfatiza que, mesmo em relação às atividades ininterruptas, como é o caso de um hipermercado, “há que prevalecer o direito irrestrito de acesso às instalações sanitárias da empresa, durante a jornada de trabalho”. Por isso, também a contagem do número de pausas e do tempo despendido em cada uma delas “em si mesma já constitui intolerável constran

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [ ]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n', additional_kwargs={}, response_metadata={})]


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [ ]:
for chunk in rag_chain.stream("Qual é a empresa denunciada?"):
    print(chunk, end="", flush=True)

A empresa denunciada é a Companhia Zaffari. Os funcionários relataram condições de trabalho degradantes e violações de direitos.